In [148]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB


In [149]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_hero = pd.read_json('data/heros.json', orient='records')

In [150]:
df_hero = df_hero['heroes']

In [151]:
df_hero

0      {'name': 'antimage', 'id': 1, 'localized_name'...
1      {'name': 'axe', 'id': 2, 'localized_name': 'Axe'}
2      {'name': 'bane', 'id': 3, 'localized_name': 'B...
3      {'name': 'bloodseeker', 'id': 4, 'localized_na...
4      {'name': 'crystal_maiden', 'id': 5, 'localized...
                             ...                        
107    {'name': 'phoenix', 'id': 110, 'localized_name...
108    {'name': 'techies', 'id': 105, 'localized_name...
109    {'name': 'oracle', 'id': 111, 'localized_name'...
110    {'name': 'winter_wyvern', 'id': 112, 'localize...
111    {'name': 'arc_warden', 'id': 113, 'localized_n...
Name: heroes, Length: 112, dtype: object

In [152]:
df_train

,cluster_id,game_mode,game_type,hero_1,hero_2,hero_3,hero_4,hero_5,hero_6,hero_7,...,hero_105,hero_106,hero_107,hero_108,hero_109,hero_110,hero_111,hero_112,hero_113,outcome
0,223,2,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,team_2
1,152,2,2,0,0,0,1,0,-1,0,...,0,0,0,0,0,0,0,0,0,team_1
2,131,2,2,0,0,0,1,0,-1,0,...,0,0,0,0,0,0,0,0,0,team_1
3,154,2,2,0,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,team_1
4,171,2,3,0,0,0,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,team_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92645,154,2,3,1,0,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,team_2
92646,154,2,2,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,team_1
92647,111,2,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,team_1
92648,185,2,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,team_2


In [153]:
def dataFilter(data):
    
    data = df_train.drop(["cluster_id", "game_mode", "game_type"], axis=1)
    features = data.drop("outcome", axis=1)
    classes = data.outcome
    normalize = []
    for i in classes:
        if(i == "team_2"):
            normalize.append(1)
        else:
            normalize.append(-1)
    return np.array(features), normalize

In [154]:
# Data preparation

df_train_features, train_classes = dataFilter(df_train)
df_test_features, test_classes = dataFilter(df_test)

In [156]:
np.array(df_train_features)[1]

array([ 0,  0,  0,  1,  0, -1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0, -1,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,
       -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [157]:
train_classes[1]

-1

In [159]:
def buildClassifier(train_data):
    features, classes = dataFilter(train_data)
    # create naive bayes classifier
    gaunb = GaussianNB()
    gaunb = gaunb.fit(features, classes)
    return gaunb

In [166]:
predictor = buildClassifier(df_train)

In [167]:
#testing predictor functionality
predictor.predict(np.array(df_train_features))

array([-1, -1, -1, ..., -1,  1, -1])

In [168]:
#analysis of predictor with its own train data

from sklearn.metrics import accuracy_score
predict_result = predictor.predict(np.array(df_train_features))
score = accuracy_score(classes, predict_result)
print(score)

0.5729519697787372


In [169]:
#analysis of predictor with its test data

predict_result_test = predictor.predict(np.array(df_test_features))
score_test = accuracy_score(test_classes, predict_result_test)
print(score_test)

0.5729519697787372


In [170]:
def getMyLastPick(picks, clf):
    possiblePicks = []
    for i in range(1,114):
        trial = picks.copy()
        if(i not in picks):
            trial.append(i)
            result = clf.predict(trial)
            if(result == np.array(1)):
                possiblePicks.append(trial)
    return possiblePicks

In [158]:
#return last pick given (1,2,3,4,5,6,7,8,9) [1,2,3,4,5] [6,7,8,9,?] <-- your team
#for i in range(1,114) try everything but cannot pick currently selected heroes, and find best

In [171]:
def getHeroNames(draft):
    #link to json with id
    return 0